In [ ]:
!pip install catboost

In [ ]:
from sentence_transformers import SentenceTransformer
import torch
from transformers import AutoProcessor, AutoModel
from PIL import Image
import pandas as pd
import polars as pl
from tqdm import tqdm

import numpy as np
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import average_precision_score, make_scorer
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier, Pool

In [ ]:
train_df = pl.scan_parquet('/content/drive/MyDrive/data_second_stage/accessories_train.parquet').drop(["characteristics1", "characteristics2", "parentname1", "parentname2"])
train_df_pd = train_df.collect().to_pandas()
train_df_pd

,id,id1,subjectname1,title1,description1,id2,subjectname2,title2,description2,target
0,a5591ad3a84d2246382c,a5591ad3a8,Зарядные устройства,Длинная зарядка для iPhone 20W,Невероятно быстрая зарядка для IPhone! Предста...,4d2246382c,Зарядные устройства,Быстрая зарядка для iPhone 20w Type-C Lightning,Молниеносная быстрая зарядка для iPhone с Type...,False
1,e6a2a5a212e3df58b85c,e6a2a5a212,Защитные стекла,Защитное стекло для iPhone 12 / 12 PRO,Противоударное защитное стекло 3D Medicine Gla...,e3df58b85c,Защитные стекла,Защитное стекло на iPhone 12 / 12 Pro Remax 2 шт,Противоударное ультратонкое олеофобное стекло ...,False
2,991288750fe029c5ec0f,991288750f,Чехлы для телефонов,Чехол на Samsung A55,Ищете идеальный чехол для вашего Samsung Galax...,e029c5ec0f,Чехлы для телефонов,Чехол на Samsung A55 прозрачный с защитой камеры,Чехол на Samsung Galaxy A55 является надежным ...,False
3,24d0262006f8e5bd35ab,24d0262006,Чехлы для телефонов,Чехол на Samsung S23 Ultra для самсунг с23 ультра,Защитный силиконовый чехол для смартфона Samsu...,f8e5bd35ab,Чехлы для телефонов,Чехол на Samsung S23 Ultra,Чехол для Samsung Galaxy S23 Ultra-это отлично...,False
4,fc37b893e45693f66f84,fc37b893e4,Чехлы для телефонов,Чехол на iPhone 11 прозрачный. Комплект 5 цветов,В комплекте сразу 5 чехла разного цвета! Это о...,5693f66f84,Чехлы для телефонов,Чехол на iPhone 11 прозрачный. Комплект 5 цветов,В комплекте сразу 5 чехла разного цвета! Это о...,True
...,...,...,...,...,...,...,...,...,...,...
39827,050dc7cac50acb423292,050dc7cac5,Защитные стекла,Защитное стекло на iPhone 13 / 13 pro / 14,Противоударное защитное стекло Remax 3D Medici...,0acb423292,Защитные стекла,Защитное стекло GL-27 Remax на Айфон 13/13 Pro/14,Противоударное защитное стекло Remax 3D Medici...,True
39828,952a4cd4f49541414017,952a4cd4f4,Чехлы для телефонов,Чехол на Realme C55,Яркий стильный оригинальный силиконовый чехол ...,9541414017,Чехлы для телефонов,Чехол на Infinix Hot 30,ВНИМАНИЕ!!! Чехол подходит только для INFINIX ...,False
39829,404d61afe91ca4e0b58b,404d61afe9,Чехлы для телефонов,Чехол для iPhone 11 защитный с принтом,Матовый чехол со змеями на iPhone 11 обеспечит...,1ca4e0b58b,Чехлы для телефонов,Чехол на iphone 13,Ищете трендовый и надежный способ защитить сво...,False
39830,4b4395024ed45e1a5a21,4b4395024e,Защитные стекла,Защитное стекло на Redmi 10,Представьте себе идеальную гармонию — прозрачн...,d45e1a5a21,Защитные стекла,"Защитное стекло на Poco X3 Pro, Redmi note 9 p...",Представьте себе идеальную гармонию — прозрачн...,False


In [ ]:
test_df = pl.scan_parquet('/content/drive/MyDrive/data_second_stage/accessories_test.parquet').drop(["characteristics1", "characteristics2", "parentname1", "parentname2"])
test_df_pd = test_df.collect().to_pandas()
test_df_pd

,id,id1,subjectname1,title1,description1,id2,subjectname2,title2,description2
0,1dcb4a9e89eeb1486b16,1dcb4a9e89,Чехлы для телефонов,Чехол на iPhone 15 Plus с принтом,Чехол для iPhone 15 Plus сделан из высококачес...,eeb1486b16,Чехлы для телефонов,Чехол на iPhone 15 Plus с бантиками,Чехол на iPhone 15 Plus не теряет своей формы ...
1,a2c425dc439330516d91,a2c425dc43,Чехлы для телефонов,Силиконовый чехол на iPhone 13 с защитой камеры,"Чехол создан для тех, кто ценит элегантность,...",9330516d91,Чехлы для телефонов,Чехол на iPhone 13 / айфон 13 яркий однотонный,Чехол для Iphone 13 — стильный и функциональны...
2,4de52efbfce684de6a1a,4de52efbfc,Дисплеи для смартфонов,"Дисплей для Honor Lite 10i, 20i, 20e в рамке",LCD дисплей для телефона Honor 10 Lite/10i/20i...,e684de6a1a,Дисплеи для смартфонов,Дисплей для Huawei Honor 10 Lite/10i/20i/20e (...,Дисплей для Huawei Honor 10 Lite/10i/20i/20e (...
3,3007c60fb56f701cdaa4,3007c60fb5,Чехлы для телефонов,Силиконовый чехол на Infinix Zero 30 4G Инфини...,Волнистый чехол для Infinix Zero 30 4G с плотн...,6f701cdaa4,Чехлы для телефонов,Чехол на Infinix Zero 30 4G с рисунком,Силиконовый чехол на телефон Infinix Zero 30 4...
4,5202734262e3ad298717,5202734262,Чехлы для телефонов,Чехол на Oppo A38 / Oppo A18 с принтом,Чехол на Oppo A38 / Oppo A18 не теряет своей ф...,e3ad298717,Чехлы для телефонов,Чехол на Oppo A38 / Oppo A18 с принтом,Чехол для Oppo A38 / Oppo A18 сделан из высоко...
...,...,...,...,...,...,...,...,...,...
10092,25b565d8853928b03cc7,25b565d885,Зарядные устройства,"Зарядное устройство для телефона, быстрая type...",зарядка type-c 67w Xiaomi для телефона с подде...,3928b03cc7,Зарядные устройства,Зарядка type-c 67w быстрая,Зарядное устройство для телефона с кабелем USB...
10093,5594371ffb3d052a1d96,5594371ffb,Чехлы для телефонов,Чехол-книжка MyPads Premium для Samsung Galaxy...,Чехол-книжка MyPads Premium для Samsung Galaxy...,3d052a1d96,Чехлы для телефонов,Чехол-книжка MyPads Premium для Samsung Galaxy...,Чехол-книжка MyPads Premium для Samsung Galaxy...
10094,bf00b2f14c54f12e77a6,bf00b2f14c,Чехлы для телефонов,Чехол на Samsung Galaxy A33 5G,Силиконовый чехол для Samsung Galaxy A33 5G с ...,54f12e77a6,Чехлы для телефонов,Чехол на Samsung A33 5G прикольный,Силиконовый чехол на Samsung Galaxy A33 5G с п...
10095,ddda09d6f6752f19d182,ddda09d6f6,Зарядные устройства,Зарядное устройство Сяоми 67W и кабель USB - T...,Зарядка Xiaomi 67W в комплекте с кабелем USB –...,752f19d182,Зарядные устройства,Быстрая зарядка для телефона Xiaomi 67w Type-c,Сетевой адаптер для Xiaomi 67W в комплекте с к...


## Косинусная близость текстов

In [ ]:
text_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def get_text_embedding(text: str):
  emb = text_model.encode(text, convert_to_tensor=True, normalize_embeddings=True)
  return emb



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def cosine_similarity(emb1, emb2):
  return torch.nn.functional.cosine_similarity(
    emb1.unsqueeze(0),
    emb2.unsqueeze(0)
  ).item()

In [ ]:
def get_text_sim(text1, text2):
  t1 = get_text_embedding(text1)
  t2 = get_text_embedding(text2)
  text_sim = cosine_similarity(t1, t2)
  return text_sim

In [ ]:
def getstr(characteristics):
  chars = []
  for char in characteristics:
    if char not in chars:
      chars.append(char)

  charstr = ''
  for char in chars:
    charstr = charstr + char['charcName'] + ': '
    if char['value'] != None:
      charstr = charstr + str(char['value']) + '. '
    elif char['charcValues'] != None:
      charstr = charstr + char['charcValues'] + '. '

  return charstr

In [ ]:
subjectname_sim = []
title_sim = []
description_sim = []
characteristics_sim = []

### train и test

In [ ]:
for i in range(3):
  train_df_chunk = pl.scan_parquet('/content/drive/MyDrive/data_second_stage/house_full_train.parquet').slice(i*10000,10000)
  train_df_chunk_pd = train_df_chunk.collect().to_pandas()

  subjectname1 = train_df_chunk_pd['subjectname1'].tolist()
  subjectname2 = train_df_chunk_pd['subjectname2'].tolist()
  title1 = train_df_chunk_pd['title1'].tolist()
  title2 = train_df_chunk_pd['title2'].tolist()
  description1 = train_df_chunk_pd['description1'].tolist()
  description2 = train_df_chunk_pd['description2'].tolist()
  characteristics1 = train_df_chunk_pd['characteristics1'].tolist()
  characteristics2 = train_df_chunk_pd['characteristics2'].tolist()


  for j in tqdm(range(10000)):
    subjectname_sim.append(get_text_sim(subjectname1[j], subjectname2[j]))
    title_sim.append(get_text_sim(title1[j], title2[j]))
    description_sim.append(get_text_sim(description1[j], description2[j]))

    chrs1 = getstr(characteristics1[j])
    chrs2 = getstr(characteristics2[j])

    characteristics_sim.append(get_text_sim(chrs1, chrs2))

  print(len(subjectname_sim))




100%|██████████| 10000/10000 [11:12<00:00, 14.87it/s]


10000


100%|██████████| 10000/10000 [10:34<00:00, 15.76it/s]


20000


100%|██████████| 10000/10000 [10:33<00:00, 15.78it/s]

30000


In [ ]:
len(subjectname_sim)

30000

In [ ]:
train_df_chunk = pl.scan_parquet('/content/drive/MyDrive/data_second_stage/house_full_train.parquet').slice(30000,9649)
train_df_chunk_pd = train_df_chunk.collect().to_pandas()

subjectname1 = train_df_chunk_pd['subjectname1'].tolist()
subjectname2 = train_df_chunk_pd['subjectname2'].tolist()
title1 = train_df_chunk_pd['title1'].tolist()
title2 = train_df_chunk_pd['title2'].tolist()
description1 = train_df_chunk_pd['description1'].tolist()
description2 = train_df_chunk_pd['description2'].tolist()
characteristics1 = train_df_chunk_pd['characteristics1'].tolist()
characteristics2 = train_df_chunk_pd['characteristics2'].tolist()

for j in tqdm(range(9649)):
  subjectname_sim.append(get_text_sim(subjectname1[j], subjectname2[j]))
  title_sim.append(get_text_sim(title1[j], title2[j]))
  description_sim.append(get_text_sim(description1[j], description2[j]))

  chrs1 = getstr(characteristics1[j])
  chrs2 = getstr(characteristics2[j])

  characteristics_sim.append(get_text_sim(chrs1, chrs2))

len(subjectname_sim)

100%|██████████| 9649/9649 [10:18<00:00, 15.60it/s]


39649

In [ ]:
train_df_pd['subjectname_sim'] = subjectname_sim
train_df_pd['title_sim'] = title_sim
train_df_pd['description_sim'] = description_sim
train_df_pd['characteristics_sim'] = characteristics_sim

# train_df_pd.to_parquet('/content/drive/MyDrive/wb/acc_train_simple.parquet')
# train_df_pd

In [ ]:
len(subjectname_sim)

0

In [ ]:
# for i in range(1):
#   test_df_chunk = pl.scan_parquet('/content/drive/MyDrive/data_second_stage/house_test.parquet').slice(i*10000,10000)
#   test_df_chunk_pd = test_df_chunk.collect().to_pandas()

#   subjectname1 = test_df_chunk_pd['subjectname1'].tolist()
#   subjectname2 = test_df_chunk_pd['subjectname2'].tolist()
#   title1 = test_df_chunk_pd['title1'].tolist()
#   title2 = test_df_chunk_pd['title2'].tolist()
#   description1 = test_df_chunk_pd['description1'].tolist()
#   description2 = test_df_chunk_pd['description2'].tolist()
#   characteristics1 = test_df_chunk_pd['characteristics1'].tolist()
#   characteristics2 = test_df_chunk_pd['characteristics2'].tolist()


#   for j in tqdm(range(10000)):
#     subjectname_sim.append(get_text_sim(subjectname1[j], subjectname2[j]))
#     title_sim.append(get_text_sim(title1[j], title2[j]))
#     description_sim.append(get_text_sim(description1[j], description2[j]))

#     chrs1 = getstr(characteristics1[j])
#     chrs2 = getstr(characteristics2[j])

#     characteristics_sim.append(get_text_sim(chrs1, chrs2))

#   print(len(subjectname_sim))


test_df_chunk = pl.scan_parquet('/content/drive/MyDrive/data_second_stage/house_full_test.parquet')
test_df_chunk_pd = test_df_chunk.collect().to_pandas()

subjectname1 = test_df_chunk_pd['subjectname1'].tolist()
len(subjectname1)
subjectname2 = test_df_chunk_pd['subjectname2'].tolist()
title1 = test_df_chunk_pd['title1'].tolist()
title2 = test_df_chunk_pd['title2'].tolist()
description1 = test_df_chunk_pd['description1'].tolist()
description2 = test_df_chunk_pd['description2'].tolist()
characteristics1 = test_df_chunk_pd['characteristics1'].tolist()
characteristics2 = test_df_chunk_pd['characteristics2'].tolist()

# for j in tqdm(range(9909)):
#   subjectname_sim.append(get_text_sim(subjectname1[j], subjectname2[j]))
#   title_sim.append(get_text_sim(title1[j], title2[j]))
#   description_sim.append(get_text_sim(description1[j], description2[j]))

#   chrs1 = getstr(characteristics1[j])
#   chrs2 = getstr(characteristics2[j])

#   characteristics_sim.append(get_text_sim(chrs1, chrs2))

# test_df_pd['subjectname_sim'] = subjectname_sim
# test_df_pd['title_sim'] = title_sim
# test_df_pd['description_sim'] = description_sim
# test_df_pd['characteristics_sim'] = characteristics_sim

### Картинки

In [ ]:
def webp_to_vector(image_path, model, target_size=(5, 5)):
    """
    Преобразует WebP изображение в вектор-эмбеддинг.

    Args:
        image_path (str): Путь к изображению в формате WebP
        model: Модель для извлечения признаков (если None, используется упрощенный подход)
        target_size (tuple): Размер для ресайза (ширина, высота)

    Returns:
        torch.Tensor: Векторное представление изображения
    """
    # 1. Загрузка и предобработка изображения
    image = Image.open(image_path).convert('RGB')

    # 2. Преобразования: ресайз и нормализация
    transform = transforms.Compose([
        transforms.Resize(target_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    # Применяем преобразования
    image_tensor = transform(image).unsqueeze(0)  # добавляем batch dimension


    # 4. Извлечение признаков
    with torch.no_grad():
        features = model(image_tensor)

    return features.squeeze()

def img_cosine_similarity(vec1, vec2):
    """
    Вычисляет косинусную близость между двумя векторами.

    Args:
        vec1, vec2 (torch.Tensor): Входные векторы

    Returns:
        float: Косинусная близость в диапазоне [-1, 1]
    """
    # Нормализуем векторы
    vec1_norm = vec1 / torch.norm(vec1)
    vec2_norm = vec2 / torch.norm(vec2)

    # Вычисляем косинусную близость
    similarity = torch.dot(vec1_norm, vec2_norm)

    return similarity.item()


# Пути к изображениям
def get_img_sim(image1_path, image2_path, model):

    # image1_path = "images\\beauty\\0.webp"
    # image2_path = "images\\beauty\\0a0a40cc6c.webp"

    try:
        # Преобразуем изображения в векторы
        vec1 = webp_to_vector(image1_path, model=model)
        vec2 = webp_to_vector(image2_path, model=model)

        # print(f"Размер вектора 1: {vec1.shape}")
        # print(f"Размер вектора 2: {vec2.shape}")

        # Вычисляем косинусную близость
        return img_cosine_similarity(vec1, vec2)

    except FileNotFoundError as e:
        print(f"Ошибка: Файл не найден - {e}")
        return 0
    # except Exception as e:
    #     print(f"Произошла ошибка:

In [ ]:
class SimpleFeatureExtractor(nn.Module):
    def __init__(self):
        super(SimpleFeatureExtractor, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))
        )
    def forward(self, x):
        x = self.features(x)
        return x.flatten(1)

model = SimpleFeatureExtractor()
model.eval()

id1 = train_df_pd['id1'].tolist()
id2 = train_df_pd['id2'].tolist()

image_sim = []

for i in tqdm(range(9909)):
    path1 =  f"images/house/{id1[i]}.webp"
    path2 =  f"images/house/{id2[i]}.webp"

    image_sim.append(get_img_sim(path1, path2, model))

In [ ]:
df_img = pl.scan_parquet('/content/drive/MyDrive/data_second_stage/acc_test_img.parquet')
df_img_pd = df_img.collect().to_pandas()

img_sim = df_img_pd['img_sim'].tolist()
# train_df_pd['img_sim'] = img_sim
test_df_pd['img_sim'] = img_sim


In [ ]:
test_df_pd

,id,id1,subjectname1,title1,description1,id2,subjectname2,title2,description2,img_sim
0,1dcb4a9e89eeb1486b16,1dcb4a9e89,Чехлы для телефонов,Чехол на iPhone 15 Plus с принтом,Чехол для iPhone 15 Plus сделан из высококачес...,eeb1486b16,Чехлы для телефонов,Чехол на iPhone 15 Plus с бантиками,Чехол на iPhone 15 Plus не теряет своей формы ...,0.982173
1,a2c425dc439330516d91,a2c425dc43,Чехлы для телефонов,Силиконовый чехол на iPhone 13 с защитой камеры,"Чехол создан для тех, кто ценит элегантность,...",9330516d91,Чехлы для телефонов,Чехол на iPhone 13 / айфон 13 яркий однотонный,Чехол для Iphone 13 — стильный и функциональны...,0.912307
2,4de52efbfce684de6a1a,4de52efbfc,Дисплеи для смартфонов,"Дисплей для Honor Lite 10i, 20i, 20e в рамке",LCD дисплей для телефона Honor 10 Lite/10i/20i...,e684de6a1a,Дисплеи для смартфонов,Дисплей для Huawei Honor 10 Lite/10i/20i/20e (...,Дисплей для Huawei Honor 10 Lite/10i/20i/20e (...,0.910959
3,3007c60fb56f701cdaa4,3007c60fb5,Чехлы для телефонов,Силиконовый чехол на Infinix Zero 30 4G Инфини...,Волнистый чехол для Infinix Zero 30 4G с плотн...,6f701cdaa4,Чехлы для телефонов,Чехол на Infinix Zero 30 4G с рисунком,Силиконовый чехол на телефон Infinix Zero 30 4...,0.912185
4,5202734262e3ad298717,5202734262,Чехлы для телефонов,Чехол на Oppo A38 / Oppo A18 с принтом,Чехол на Oppo A38 / Oppo A18 не теряет своей ф...,e3ad298717,Чехлы для телефонов,Чехол на Oppo A38 / Oppo A18 с принтом,Чехол для Oppo A38 / Oppo A18 сделан из высоко...,0.600306
...,...,...,...,...,...,...,...,...,...,...
10092,25b565d8853928b03cc7,25b565d885,Зарядные устройства,"Зарядное устройство для телефона, быстрая type...",зарядка type-c 67w Xiaomi для телефона с подде...,3928b03cc7,Зарядные устройства,Зарядка type-c 67w быстрая,Зарядное устройство для телефона с кабелем USB...,0.886438
10093,5594371ffb3d052a1d96,5594371ffb,Чехлы для телефонов,Чехол-книжка MyPads Premium для Samsung Galaxy...,Чехол-книжка MyPads Premium для Samsung Galaxy...,3d052a1d96,Чехлы для телефонов,Чехол-книжка MyPads Premium для Samsung Galaxy...,Чехол-книжка MyPads Premium для Samsung Galaxy...,0.695297
10094,bf00b2f14c54f12e77a6,bf00b2f14c,Чехлы для телефонов,Чехол на Samsung Galaxy A33 5G,Силиконовый чехол для Samsung Galaxy A33 5G с ...,54f12e77a6,Чехлы для телефонов,Чехол на Samsung A33 5G прикольный,Силиконовый чехол на Samsung Galaxy A33 5G с п...,0.984017
10095,ddda09d6f6752f19d182,ddda09d6f6,Зарядные устройства,Зарядное устройство Сяоми 67W и кабель USB - T...,Зарядка Xiaomi 67W в комплекте с кабелем USB –...,752f19d182,Зарядные устройства,Быстрая зарядка для телефона Xiaomi 67w Type-c,Сетевой адаптер для Xiaomi 67W в комплекте с к...,0.999075


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Catboost

In [ ]:
categorical_features = ['subjectname1', 'subjectname2']
text_features = ['title1', 'title2', 'description1', 'description2']

In [ ]:
# Разделение на признаки и целевую переменную
X = train_df_pd.drop(columns=["target", "id", "id1", "id2"])
y = train_df_pd['target']

# Разделение на train/val
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.1, random_state=42, stratify=y
)

X_test = test_df_pd.drop(columns=["id", "id1", "id2"])


In [ ]:
train_pool = Pool(
    data=X_train,
    label=y_train,
    cat_features=categorical_features,
    text_features=text_features
)

val_pool = Pool(
    data=X_val,
    label=y_val,
    cat_features=categorical_features,
    text_features=text_features
)

test_pool = Pool(
    data=X_test,
    cat_features=categorical_features,
    text_features=text_features
)

### Подбор гиперпараметров

In [ ]:
average_precision_scorer = make_scorer(
    average_precision_score,
    response_method='predict_proba',
    greater_is_better=True
)

In [ ]:
base_model = CatBoostClassifier(
    cat_features=categorical_features,
    text_features=text_features,
    random_seed=42,
    verbose=0,
    task_type='GPU',
    eval_metric='PRAUC',
    early_stopping_rounds=50
)

In [ ]:
param_dist = {
    'iterations': [100],
    'depth': [3, 6, 8],
    'learning_rate': [0.01, 0.1, 0.3],
    'l2_leaf_reg': [1, 5, 9],
    'bagging_temperature': [0, 1.0, 2.0],
    'random_strength': [1, 3, 5],
    'border_count': [32, 150, 254],
    'min_data_in_leaf': [1, 10, 20],
    'grow_policy': ['SymmetricTree', 'Depthwise', 'Lossguide'],
    'bootstrap_type': ['Bayesian']
}

In [ ]:
random_search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_dist,
    n_iter=3,                     # Количество случайных комбинаций
    scoring=average_precision_scorer,  # Наша метрика
    cv=3,                          # Количество фолдов кросс-валидации
    n_jobs=1,                      # Для Pool используем 1 job
    verbose=2,                     # Детальный вывод прогресса
    random_state=42,               # Для воспроизводимости
)

In [ ]:
random_search.fit(
    X_train, y_train,           # Данные для обучения
    eval_set=(X_val, y_val),    # Данные для валидации
    verbose=0                   # Отключаем вывод CatBoost
)

print("\nЛУЧШИЕ ПАРАМЕТРЫ:")
for param, value in random_search.best_params_.items():
    print(f"  {param}: {value}")

# print(f"\nЛучший Average Precision Score (кросс-валидация): {random_search.best_score_:.4f}")

Fitting 3 folds for each of 3 candidates, totalling 9 fits


Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


[CV] END bagging_temperature=0, bootstrap_type=Bayesian, border_count=150, depth=3, grow_policy=Depthwise, iterations=100, l2_leaf_reg=5, learning_rate=0.3, min_data_in_leaf=10, random_strength=5; total time=  13.7s


Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


[CV] END bagging_temperature=0, bootstrap_type=Bayesian, border_count=150, depth=3, grow_policy=Depthwise, iterations=100, l2_leaf_reg=5, learning_rate=0.3, min_data_in_leaf=10, random_strength=5; total time=  13.8s


...

[CV] END bagging_temperature=2.0, bootstrap_type=Bayesian, border_count=150, depth=3, grow_policy=Depthwise, iterations=100, l2_leaf_reg=5, learning_rate=0.1, min_data_in_leaf=20, random_strength=1; total time=  13.6s


Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


[CV] END bagging_temperature=2.0, bootstrap_type=Bayesian, border_count=150, depth=3, grow_policy=Depthwise, iterations=100, l2_leaf_reg=5, learning_rate=0.1, min_data_in_leaf=20, random_strength=1; total time=  13.2s


Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time



ЛУЧШИЕ ПАРАМЕТРЫ:
  random_strength: 5
  min_data_in_leaf: 10
  learning_rate: 0.3
  l2_leaf_reg: 5
  iterations: 100
  grow_policy: Depthwise
  depth: 3
  border_count: 150
  bootstrap_type: Bayesian
  bagging_temperature: 0


### Лучшая модель

In [ ]:
best_model = CatBoostClassifier(
    cat_features=categorical_features,
    text_features=text_features,
    random_seed=42,
    verbose=100,
    task_type='GPU',
    eval_metric='PRAUC',
    random_strength = 5,
    min_data_in_leaf = 10,
    learning_rate = 0.1,
    iterations=1500,
    grow_policy = 'Depthwise',
    depth=3,
    border_count = 150,
    bootstrap_type = 'Bayesian',
    bagging_temperature = 0

)

best_model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=100)
y_pred_proba = best_model.predict_proba(X_val)[:, 1]
ap_score = average_precision_score(y_val, y_pred_proba)
print(f"Average Precision на валидации: {ap_score:.4f}")



Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.7033921	test: 0.7219066	best: 0.7219066 (0)	total: 29.3ms	remaining: 43.9s
100:	learn: 0.7799519	test: 0.7710158	best: 0.7710158 (100)	total: 4.49s	remaining: 1m 2s
200:	learn: 0.8067840	test: 0.7840944	best: 0.7840944 (200)	total: 5.87s	remaining: 37.9s
300:	learn: 0.8252935	test: 0.7932782	best: 0.7932782 (300)	total: 7.23s	remaining: 28.8s
400:	learn: 0.8405548	test: 0.7995422	best: 0.7995422 (400)	total: 8.79s	remaining: 24.1s
500:	learn: 0.8520539	test: 0.8033675	best: 0.8034678 (495)	total: 10.2s	remaining: 20.4s
600:	learn: 0.8618911	test: 0.8052005	best: 0.8056280 (585)	total: 11.6s	remaining: 17.3s
700:	learn: 0.8704063	test: 0.8066863	best: 0.8067229 (690)	total: 13s	remaining: 14.8s
800:	learn: 0.8776045	test: 0.8079616	best: 0.8080424 (780)	total: 17.1s	remaining: 14.9s
900:	learn: 0.8840687	test: 0.8094726	best: 0.8094983 (895)	total: 18.7s	remaining: 12.4s
1000:	learn: 0.8907361	test: 0.8128655	best: 0.8128655 (1000)	total: 20.3s	remaining: 10.1s
1100:	learn: 

In [ ]:
test_prediction = best_model.predict_proba(test_pool)[:, 1]

In [ ]:
sub_df = test_df.select("id").collect().to_pandas()
sub_df.loc[:, "y_pred"] = test_prediction
sub_df.to_csv("/content/drive/MyDrive/wb/acc1_submission.csv", index=False)
sub_df.head()

,id,y_pred
0,1dcb4a9e89eeb1486b16,0.077267
1,a2c425dc439330516d91,0.934304
2,4de52efbfce684de6a1a,0.496692
3,3007c60fb56f701cdaa4,0.035724
4,5202734262e3ad298717,0.069475


In [ ]:
subdf2 = pd.read_csv('/content/drive/MyDrive/wb/house_submission.csv')
subdf1 = pd.read_csv('/content/drive/MyDrive/wb/acc_submission.csv')
df_new = pd.concat([subdf1, subdf2], ignore_index=True)

df_new.to_csv("/content/drive/MyDrive/wb/final1_submission.csv", index=False)

In [ ]:
df_new


,id,y_pred
0,1dcb4a9e89eeb1486b16,0.074469
1,a2c425dc439330516d91,0.823322
2,4de52efbfce684de6a1a,0.375011
3,3007c60fb56f701cdaa4,0.034523
4,5202734262e3ad298717,0.134243
...,...,...
20001,1f320eec3231c7e5cb28,0.248323
20002,29925d3ae9b56b70d0eb,0.966936
20003,dd6c4edfdc02ce5f900a,0.981000
20004,a6a0c997f2f30ae0647b,0.984419
